## 通过深度学习框架简洁地实现线性回归模型

In [112]:
import numpy as np
import torch
from torch.utils import data 
from d2l import torch as d2l

1.创建数据

In [113]:
true_w = torch.tensor([2,-3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000) # 生成数据
# features.shape (1000,2)  labels.shape (1000,1)

2.利用TensorDataset和DataLoader自动创建生成小批量数据

In [114]:
def load_array(features, labels, batch_size, is_train=True):
    # 构造一个PyTorch数据迭代器
    dataset = data.TensorDataset(features, labels)
    # 直接用 TensorDataset() 将tensor类型的数据x_data和y_data包装成Dataset类，便于后续作为DataLoader的参数

    return data.DataLoader(dataset, batch_size, shuffle=is_train) # 每次从中随机挑选batch_size个样本
    # 只对训练集进行shuffle 测试集不shuffle
    # TensorDataset将输入的两类数据一一对应；DataLoader将两类数据重新排序

batch_size = 10
data_iter = load_array(features, labels, batch_size) # 此处直接调用torch.utils中的data库，生成类似迭代器
# type(data_iter) # torch.utils.data.dataloader.DataLoader
next(iter(data_iter)) # 转换为Python的可迭代对象; 并输出一个; 也可用for循环输出

[tensor([[-1.6334,  0.0691],
         [ 1.4405, -0.8137],
         [-1.3261,  0.8369],
         [ 0.1328,  1.6695],
         [-0.7945,  0.4001],
         [-0.3057,  1.3405],
         [-0.9769, -0.6522],
         [-0.2217,  0.6948],
         [ 1.2154,  2.3783],
         [ 0.4641, -0.3715]]),
 tensor([[ 0.6994],
         [ 9.8418],
         [-1.3081],
         [-1.2115],
         [ 1.2565],
         [-0.9617],
         [ 4.4657],
         [ 1.4059],
         [-1.4456],
         [ 6.3758]])]

3. 定义模型

In [115]:
from torch import nn # neural network 定义了大量的层

net = nn.Sequential(nn.Linear(2,1)) # 用Linear层(线性层/全连接层)；输入维度2 输出维度1
# 然后将Linear层放到Sequential容器中，该容器理解为List of layers，将层按序放在一起

In [116]:
net[0].weight # 会被自动默认初始化为服从均值分布

Parameter containing:
tensor([[0.6502, 0.4347]], requires_grad=True)

4. 定义初始化模型参数

In [117]:
# net[0] # net[0]=Linear(in_features=2, out_features=1, bias=True)
# net[0].bias是张量b，net[0].weight是张量w
net[0].weight.data.normal_(0,0.01) # 初始化
net[0].bias.data.fill_(0) # 偏差设为0

tensor([0.])

5. 使用内置的均方误差MSELoss和SGD

In [118]:
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.03) # net.parameters()是个generator包含所有的参数w和b

6. 训练过程

In [119]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        # Forward
        y_pred = net(X)
        loss = criterion(y_pred, y)
        # Backward
        optimizer.zero_grad() # 先清零优化器的梯度
        loss.backward() # pytorch自动进行了sum
        # Updata
        optimizer.step() # 更新模型；优雅
    loss = criterion(net(features), labels)
    print(f'epoch {epoch+1}, train_loss {loss:f}')

epoch 1, train_loss 0.000271
epoch 2, train_loss 0.000107
epoch 3, train_loss 0.000107


In [120]:
print(f'预测后的参数w: ', net[0].weight.data)
print(f'预测后的参数b: ', net[0].bias.data)

预测后的参数w:  tensor([[ 1.9998, -3.3996]])
预测后的参数b:  tensor([4.2001])
